<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention の可視化

https://jalammar.github.io/illustrated-transformer/
https://note.com/tugaa_dev/n/ne804696b46a1

In [1]:
# 必要なライブラリのインストール
!pip install transformers torch sentencepiece fugashi ipadic unidic_lite
!pip install matplotlib seaborn gradio

# 日本語フォントの設定（Gradioのグラフ内で日本語を正しく表示するため）
!apt-get -y install fonts-ipaexfont > /dev/null 2>&1 # フォントインストール
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# IPAex Gothicをフォントリストに追加
fm.fontManager.addfont('/usr/share/fonts/opentype/ipaexfont-gothic/ipaexg.ttf')

# デフォルトフォントをIPAex Gothicに設定
plt.rcParams['font.family'] = 'IPAexGothic'
plt.rcParams['font.sans-serif'] = ['IPAexGothic'] # Sans-serifフォントとしても設定

# Matplotlibのフォントキャッシュをクリア（設定を確実に適用するため）
# このコマンド実行後、Colabの「ランタイム」メニューから「ランタイムを再起動」すると、
# 日本語フォントがより確実に反映されます。
!rm -rf /root/.cache/matplotlib
print("Matplotlibの日本語フォント設定が完了しました。")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 78.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 694.9/694.9 kB 49.3 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=b563fa313957dfa8e8480e7a389f63f3ae452e8faeb9ed479598e3deb7fe45b9
  Stored in directory: /root/.cache/pip/wheels/93/8b/55/dd5978a069678c372520847cf84ba2ec539cb41917c00a2206
  Created wheel for unidic_lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658817 sha256=fbbfd2d3430e22a5b80afb13ff8897c24de61f2102f85cfac8ec4e180b4749c9
  Stored in directory: /root/.cache/pip/wheels/5e/1f/0f/4d43887e5476d956fae828ee9b6687becd5544d68b51ed633d
Successfully built ipadic unidic_lite
Matplotlibの日本語フォント設定が完了しました。


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# モデル名の指定
model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"

# トークナイザーとモデルの読み込み（Gradio関数内で再利用するためグローバル変数として定義）
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name, output_attentions=True)
model.eval() # 推論モードに設定

# Note: 「UserWarning: The secret `HF_TOKEN` does not exist in your Colab secrets.」
# という警告が表示される場合がありますが、Colab経由でHugging Face APIを叩くので
# Hugging Face Hubのトークンが設定されていないことを示しています。
# 「cl-tohoku/bert-base-japanese-whole-word-masking」は公開モデルなので、
# この警告が出てもモデルの読み込みや利用には影響ありません。ご安心ください。
# もしHugging Face Hubに自分でモデルをアップロードしたり、非公開モデルにアクセスしたりする場合は、
# トークンの設定（https://huggingface.co/settings/tokens で取得し、Colabの「Secrets」タブに設定）
# が必要になります。

# テスト用の日本語テキスト
text = "私は猫が好きです。彼は犬も好きです。"

# テキストのトークン化
inputs = tokenizer(text, return_tensors="pt")

print(f"元のテキスト: {text}")
print(f"トークン化されたID: {inputs['input_ids'].tolist()[0]}")
print(f"トークンIDを元の単語に戻す（デコード）: {tokenizer.decode(inputs['input_ids'][0])}")
print(f"Attention Mask: {inputs['attention_mask'].tolist()[0]}")

print("\n--- トークン化の可視化 ---")
tokens = tokenizer.tokenize(text)
# BERTが認識する特殊トークンを含めた完全なトークンリスト
full_tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]
print(f"完全なトークン列: {full_tokens}")

print("\n元のテキストとトークン列の対応:")
for i, token in enumerate(full_tokens):
    print(f"[{i:02d}] {token}")

print("\nトークンIDとトークンの対応:")
for i, token_id in enumerate(inputs["input_ids"][0].tolist()):
    if token_id == tokenizer.cls_token_id:
        print(f"ID: {token_id} -> トークン: [CLS]")
    elif token_id == tokenizer.sep_token_id:
        print(f"ID: {token_id} -> トークン: [SEP]")
    else:
        print(f"ID: {token_id} -> トークン: {tokenizer.decode(token_id)}")

# モデルに推論を実行し、Attention Weightを取得
with torch.no_grad(): # 勾配計算を無効化（推論のため）
    outputs = model(**inputs)

attention_weights = outputs.attentions

print(f"\nAttention layers: {len(attention_weights)}") # BERTは通常12層
print(f"Attention shape per layer (e.g., first layer): {attention_weights[0].shape}") # (1, num_heads, seq_len, seq_len)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


元のテキスト: 私は猫が好きです。彼は犬も好きです。
トークン化されたID: [2, 1325, 9, 6040, 14, 3596, 2992, 8, 306, 9, 2928, 28, 3596, 2992, 8, 3]
トークンIDを元の単語に戻す（デコード）: [CLS] 私 は 猫 が 好き です 。 彼 は 犬 も 好き です 。 [SEP]
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

--- トークン化の可視化 ---
完全なトークン列: ['[CLS]', '私', 'は', '猫', 'が', '好き', 'です', '。', '彼', 'は', '犬', 'も', '好き', 'です', '。', '[SEP]']

元のテキストとトークン列の対応:
[00] [CLS]
[01] 私
[02] は
[03] 猫
[04] が
[05] 好き
[06] です
[07] 。
[08] 彼
[09] は
[10] 犬
[11] も
[12] 好き
[13] です
[14] 。
[15] [SEP]

トークンIDとトークンの対応:
ID: 2 -> トークン: [CLS]
ID: 1325 -> トークン: 私
ID: 9 -> トークン: は
ID: 6040 -> トークン: 猫
ID: 14 -> トークン: が
ID: 3596 -> トークン: 好き
ID: 2992 -> トークン: です
ID: 8 -> トークン: 。
ID: 306 -> トークン: 彼
ID: 9 -> トークン: は
ID: 2928 -> トークン: 犬
ID: 28 -> トークン: も
ID: 3596 -> トークン: 好き
ID: 2992 -> トークン: です
ID: 8 -> トークン: 。
ID: 3 -> トークン: [SEP]

Attention layers: 12
Attention shape per layer (e.g., first layer): torch.Size([1, 12, 16, 16])


In [ ]:
import gradio as gr
import io
from PIL import Image

# tokenizerとmodelは前のセルで既に読み込まれています。

# Attention Mapとトークン化情報を生成し、画像とテキストとして返す関数
def analyze_and_plot_attention(text, layer_idx, head_idx):
    # テキストのトークン化
    inputs = tokenizer(text, return_tensors="pt")

    # 特殊トークンを含めた完全なトークンリスト
    tokens = tokenizer.tokenize(text)
    full_tokens = [tokenizer.cls_token] + tokens + [tokenizer.sep_token]

    # トークン化情報の整形
    tokenization_info = []
    tokenization_info.append(f"元のテキスト: {text}\n")
    tokenization_info.append(f"トークン化されたID: {inputs['input_ids'].tolist()[0]}\n")
    tokenization_info.append(f"トークンIDを元の単語に戻す（デコード）: {tokenizer.decode(inputs['input_ids'][0])}\n")
    tokenization_info.append(f"Attention Mask: {inputs['attention_mask'].tolist()[0]}\n")
    tokenization_info.append("\n--- トークン化の可視化 ---\n")
    tokenization_info.append(f"完全なトークン列: {full_tokens}\n")
    tokenization_info.append("\n元のテキストとトークン列の対応:\n")
    for i, token in enumerate(full_tokens):
        tokenization_info.append(f"[{i:02d}] {token}\n")
    tokenization_info.append("\nトークンIDとトークンの対応:\n")
    for i, token_id in enumerate(inputs["input_ids"][0].tolist()):
        if token_id == tokenizer.cls_token_id:
            tokenization_info.append(f"ID: {token_id} -> トークン: [CLS]\n")
        elif token_id == tokenizer.sep_token_id:
            tokenization_info.append(f"ID: {token_id} -> トークン: [SEP]\n")
        else:
            tokenization_info.append(f"ID: {token_id} -> トークン: {tokenizer.decode(token_id)}\n")

    info_text_output = "".join(tokenization_info)

    # モデルに推論を実行し、Attention Weightを取得
    with torch.no_grad():
        outputs = model(**inputs)
    attention_weights = outputs.attentions

    # 指定された層とヘッドのAttention Weightを取得
    # BERT-baseは通常12層、各層12ヘッドです
    if layer_idx >= len(attention_weights) or head_idx >= attention_weights[0].shape[1]:
        error_msg = "選択された層またはヘッドが存在しません。BERT-baseモデルは通常12層、各層12ヘッドです。"
        # エラーメッセージをトークン化情報に追加して返す
        return info_text_output + "\n" + error_msg, None

    selected_attention = attention_weights[layer_idx][0, head_idx].cpu().numpy()

    # ヒートマップの描画
    # トークン長に応じて画像サイズを動的に調整して見やすくします
    max_len = len(full_tokens)
    fig_width = max(8, max_len * 0.8) # 最小幅を8インチに設定
    fig_height = max(7, max_len * 0.7) # 最小高さを7インチに設定

    plt.figure(figsize=(fig_width, fig_height))
    sns.heatmap(selected_attention, cmap="viridis", annot=True, fmt=".2f",
                xticklabels=full_tokens, yticklabels=full_tokens,
                linewidths=.5, linecolor='lightgray', annot_kws={"size": 8}) # 注釈のフォントサイズも調整
    plt.title(f'Attention Head {head_idx} in Layer {layer_idx}', fontsize=16)
    plt.xlabel('Keys (Attended To)', fontsize=12)
    plt.ylabel('Queries (Attending From)', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout() # レイアウトを自動調整

    # 描画した画像をメモリに保存し、Gradioが扱える形式に変換
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close() # プロットを閉じてメモリを解放
    buf.seek(0) # バッファの先頭に戻る

    return info_text_output, Image.open(buf) # トークン化情報と画像をセットで返す


# Gradioインターフェースの作成
iface_attention = gr.Interface(
    fn=analyze_and_plot_attention, # 実行する関数
    inputs=[
        gr.Textbox(lines=2, placeholder="分析したい日本語テキストを入力してください...", label="入力テキスト"),
        gr.Slider(minimum=0, maximum=11, value=0, step=1, label="Transformer層 (0-11)"), # 層を選ぶスライダー
        gr.Slider(minimum=0, maximum=11, value=0, step=1, label="Attentionヘッド (0-11)") # ヘッドを選ぶスライダー
    ],
    outputs=[
        gr.Textbox(label="トークン化情報", interactive=False, lines=15), # トークン化情報を表示するテキストボックス
        gr.Image(label="Attention Map") # Attentionマップを表示する画像コンポーネント
    ],
    title="日本語BERT Attention可視化ツール",
    description="入力された日本語テキストに対し、BERTモデルのAttention機構がどのように機能しているかをヒートマップで可視化します。層とヘッドを選択し、トークン化の過程とAttentionマップの関係を観察することで、AIの思考を深く探ってみましょう"
)

# インターフェースを起動
iface_attention.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5595e389d709868d39.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
